In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[1],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

In [1]:
# sampling

import os
import os.path as pth
import json
import shutil
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

data_base_path = './data/project_deepfake'
os.makedirs(data_base_path, exist_ok=True)

train_data_path = pth.join(data_base_path, 'deepfake_1st')
test_data_path = pth.join(data_base_path, 'test')

fake_img_path_list = []
real_img_path_list = []
crop_image_percent = 0.1

label_list = [label for label in os.listdir(train_data_path)]
for label in label_list:
    label_fullpath = pth.join(train_data_path, label)
    method_list = os.listdir(label_fullpath)
    for method in method_list:
        method_fullpath = pth.join(label_fullpath, method)
        date_list = os.listdir(method_fullpath)
        for date in date_list:
            date_fullpath = pth.join(method_fullpath, date)
            name_list = os.listdir(date_fullpath)
            for name in name_list:
                name_fullpath = pth.join(date_fullpath, name)
                sub_method_list = os.listdir(name_fullpath)
                if label=='fake':
                    for sub_method in sub_method_list:
                        sub_method_fullpath = pth.join(name_fullpath, sub_method)
                        img_folder_list = os.listdir(sub_method_fullpath)                    
                        for img_folder in img_folder_list:
                            img_folder_fullpath = pth.join(sub_method_fullpath, img_folder)
                            img_list = os.listdir(img_folder_fullpath)
                            img_list = [img_folder_fullpath + '/' + img for img in img_list]
                            if int(np.ceil(len(img_list)*crop_image_percent)) >= 2:
                                crop_image_num = int(np.ceil(len(img_list)*crop_image_percent))
                            elif len(img_list) <= 1:
                                crop_image_num = 1
                            else:
                                crop_image_num = 2
                            for n in range(crop_image_num):
                                if label=='fake':
                                    fake_img_path_list.append(img_list[n])
                                else:
                                    real_img_path_list.append(img_list[n])
                else:
                    for img_folder in sub_method_list:
                        img_folder_fullpath = pth.join(name_fullpath, img_folder)
                        img_list = os.listdir(img_folder_fullpath)
                        img_list = [img_folder_fullpath + '/' + img for img in img_list]
                        if int(np.ceil(len(img_list)*crop_image_percent)) >= 2:
                            crop_image_num = int(np.ceil(len(img_list)*crop_image_percent))
                        elif len(img_list) <= 1:
                            crop_image_num = 1
                        else:
                            crop_image_num = 2
                        for n in range(crop_image_num):
                            if label=='fake':
                                fake_img_path_list.append(img_list[n])
                            else:
                                real_img_path_list.append(img_list[n])

print('-----end------')
print(fake_img_path_list[0:3])
print('<============= fake 이미지 개수: {} ==============>'.format(len(fake_img_path_list)))
print(real_img_path_list[0:3])
print('<============= real 이미지 개수: {} ==============>'.format(len(real_img_path_list)))

validation_rate = 0.2
fake_img_path_list_val = random.sample(fake_img_path_list, int(np.ceil(len(fake_img_path_list)*validation_rate)))
real_img_path_list_val = random.sample(real_img_path_list, int(np.ceil(len(real_img_path_list)*validation_rate)))

print(fake_img_path_list_val[0:3])
print('<============= fake val 이미지 개수: {} ==============>'.format(len(fake_img_path_list_val)))
print(real_img_path_list_val[0:3])
print('<============= real val 이미지 개수: {} ==============>'.format(len(real_img_path_list_val)))


-----end------
['./data/project_deepfake/deepfake_1st/fake/CW/20200902/cw-soyeon/fsgan/output_cw-soyeon_052/image_00003.jpg', './data/project_deepfake/deepfake_1st/fake/CW/20200902/cw-soyeon/fsgan/output_cw-soyeon_052/image_00006.jpg', './data/project_deepfake/deepfake_1st/fake/CW/20200902/cw-soyeon/fsgan/output_cw-soyeon_040/image_00003.jpg']
<============= fake 이미지 개수: 171651 ==============>
['./data/project_deepfake/deepfake_1st/real/CW/20201012/170878/image8/image_00043.jpg', './data/project_deepfake/deepfake_1st/real/CW/20201012/170878/image8/image_00017.jpg', './data/project_deepfake/deepfake_1st/real/CW/20201012/170878/image8/image_00021.jpg']
<============= real 이미지 개수: 66977 ==============>
['./data/project_deepfake/deepfake_1st/fake/CW/20200914/cw-hyaelin/fsgan/output_cw-hyaelin_088/image_00006.jpg', './data/project_deepfake/deepfake_1st/fake/CW/20200828/cw-yongdeok/fsgan/output_cw-yongdeok_055/image_00003.jpg', './data/project_deepfake/deepfake_1st/fake/EQ/20200815/eternal/f

In [11]:
from mtcnn import MTCNN
import cv2
import numpy as np
import matplotlib.pyplot as plt


def make_crop_img_without_detector(image_file, detector):
    result = detector.detect_faces(image_file)
    if result == []:
        print('No information')
        return []
    else:
        bounding_box = result[0]['box']
        resizeImageNDArray = image_file.copy()
        resizeImageNDArray = image_file[bounding_box[1]:bounding_box[1]+bounding_box[3], bounding_box[0]:bounding_box[0]+bounding_box[2]]
        resizeImageNDArray = cv2.resize(resizeImageNDArray, (299, 299), interpolation = cv2.INTER_CUBIC)
        return resizeImageNDArray

print('function ready')



function ready


In [18]:

detector = MTCNN()

m = 0

for i in real_img_path_list:
    images = cv2.cvtColor(cv2.imread(i), cv2.COLOR_BGR2RGB)
    images = make_crop_img_without_detector(images, detector)
    if images == []:
        print('error information in {}'.format(m))
    else:
        cv2.imwrite('./data/project_deepfake/train_new/real_samp_by_folder/{}.jpg'.format(m), cv2.cvtColor(images, cv2.COLOR_RGB2BGR))
        if m % 100 == 0:
            print(m, ' real complete')
    m += 1 
    
m = 0
for i in fake_img_path_list:
    images = cv2.cvtColor(cv2.imread(i), cv2.COLOR_BGR2RGB)
    images = make_crop_img_without_detector(images, detector)

    if images == []:
        print('error information in {}'.format(m))
    else:
        cv2.imwrite('./data/project_deepfake/train_new/fake_samp_by_folder/{}.jpg'.format(m), cv2.cvtColor(images, cv2.COLOR_RGB2BGR))
        if m % 100 == 0:
            print(m, ' fake complete')
    m += 1 

/home/ubuntu/anaconda3/envs/data_env_TF2/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


0  real complete
100  real complete
200  real complete
300  real complete
400  real complete
500  real complete
600  real complete
700  real complete
800  real complete
900  real complete
1000  real complete
No information
error information in 1054
1100  real complete
1200  real complete
1300  real complete


KeyboardInterrupt: 